In [129]:
## Importing Libraries

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import hvplot.pandas
from sqlalchemy.engine.url import URL
from sklearn.metrics import calinski_harabasz_score
from config import CONNSTRING

## Connecting to the Database
## Fetching data from the `bank_customers` table in a PostgreSQL database.

engine= create_engine(CONNSTRING)
query = "SELECT * FROM bank_customers"
df_from_sql = pd.read_sql(query, engine)
df_from_sql.head(5)




,clientnum,attrition_flag,customer_age,gender,dependent_count,education_level,marital_status,income_category,card_category,months_on_book,...,credit_limit,total_revolving_bal,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio,naive_bayes_classifier_attrition_flag_card_category_c,naive_bayes_classifier_attrition_flag_card_category_c2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777.0,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864.0,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0.0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517.0,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0.0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [130]:
## Data Cleaning
## Dropping rows with unknown values in the `income_category` column.

df_from_sql = df_from_sql[df_from_sql['income_category'] != 'Unknown']

unique_values = df_from_sql['income_category'].unique()

print(unique_values)

['$60K - $80K' 'Less than $40K' '$80K - $120K' '$40K - $60K' '$120K +']


In [131]:
## Descriptive Statistics
## Describing the dataset to understand its distribution and summary statistics.

df_from_sql.describe()

,clientnum,customer_age,dependent_count,months_on_book,total_relationship_count,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio,naive_bayes_classifier_attrition_flag_card_category_c,naive_bayes_classifier_attrition_flag_card_category_c2
count,9.015000e+03,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000,9015.000000
mean,7.393747e+08,46.332335,2.354964,35.949972,3.812313,2.337105,2.454465,8522.834764,1168.703938,7354.130826,0.759329,4419.119578,64.686966,0.711537,0.281565,0.159075,0.840925
std,3.694017e+07,7.933915,1.295146,7.923702,1.556154,1.005561,1.107845,9132.702996,813.063816,9135.277242,0.219757,3450.979966,23.681983,0.237312,0.278482,0.364446,0.364446
min,7.080821e+08,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000,0.000008,0.000462
25%,7.130482e+08,41.000000,1.000000,32.000000,3.000000,2.000000,2.000000,2505.500000,466.000000,1240.000000,0.629000,2118.000000,44.000000,0.582000,0.026000,0.000099,0.999660
50%,7.179831e+08,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4331.000000,1281.000000,3276.000000,0.736000,3868.000000,67.000000,0.702000,0.185000,0.000182,0.999820
75%,7.733865e+08,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,10808.000000,1783.500000,9574.000000,0.858000,4743.500000,81.000000,0.818000,0.516000,0.000337,0.999900
max,8.283431e+08,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,18484.000000,139.000000,3.714000,0.999000,0.999540,0.999990


In [132]:
## Categorical Columns Description
## Provide a count of unique values for each categorical feature.

print("Total categories in the feature Education_Level:\n",df_from_sql["education_level"].value_counts(), "\n")
print("Total categories in the feature Marital_Status:\n", df_from_sql["marital_status"].value_counts(), "\n")
print("Total categories in the feature Income_Category:\n", df_from_sql["income_category"].value_counts(), "\n")
print("Total categories in the feature Card_Category:\n", df_from_sql["card_category"].value_counts(), "\n")

Total categories in the feature Education_Level:
 education_level
Graduate         2796
High School      1788
Unknown          1374
Uneducated       1302
College           905
Post-Graduate     469
Doctorate         381
Name: count, dtype: int64 

Total categories in the feature Marital_Status:
 marital_status
Married     4194
Single      3499
Unknown      667
Divorced     655
Name: count, dtype: int64 

Total categories in the feature Income_Category:
 income_category
Less than $40K    3561
$40K - $60K       1790
$80K - $120K      1535
$60K - $80K       1402
$120K +            727
Name: count, dtype: int64 

Total categories in the feature Card_Category:
 card_category
Blue        8391
Silver       502
Gold         107
Platinum      15
Name: count, dtype: int64 



In [133]:
## Filtering Relevant Columns
## Select only the relevant columns for the marketing campaign analysis.

df_filtered_1 = df_from_sql[['customer_age', 'gender', 'credit_limit', 'avg_utilization_ratio','total_revolving_bal','income_category']]

df_filtered_1

,customer_age,gender,credit_limit,avg_utilization_ratio,total_revolving_bal,income_category
0,45,M,12691.0,0.061,777.0,$60K - $80K
1,49,F,8256.0,0.105,864.0,Less than $40K
2,51,M,3418.0,0.000,0.0,$80K - $120K
3,40,F,3313.0,0.760,2517.0,Less than $40K
4,40,M,4716.0,0.000,0.0,$60K - $80K
...,...,...,...,...,...,...
10122,50,M,4003.0,0.462,1851.0,$40K - $60K
10123,41,M,4277.0,0.511,2186.0,$40K - $60K
10124,44,F,5409.0,0.000,0.0,Less than $40K
10125,30,M,5281.0,0.000,0.0,$40K - $60K


In [134]:
df_filtered= df_filtered_1.copy()

In [135]:
## Data Preprocessing
## Map gender to numerical values and convert to integer

df_filtered["gender"]=df_filtered["gender"].map({"F":0,"M":1}) 

df_filtered['gender']=df_filtered['gender'].astype(int) 

In [136]:
## Demonstrating the changes done

df_filtered["gender"]

0        1
1        0
2        1
3        0
4        1
        ..
10122    1
10123    1
10124    0
10125    1
10126    0
Name: gender, Length: 9015, dtype: int32

In [137]:
## Map income category to numerical values

df_filtered["income_category"]=df_filtered["income_category"].map({"Less than $40K":0,"$40K - $60K":1,"$80K - $120K":2,"$60K - $80K":3,"$120K +":4})

## Changing the data type to integer

df_filtered['income_category']=df_filtered['income_category'].astype(int)

In [138]:
## Standardize the data

scaled_data=StandardScaler().fit_transform(df_filtered[['customer_age', 'gender', 'credit_limit', 'avg_utilization_ratio','total_revolving_bal','income_category']])
scaled_data


array([[-0.16793839,  0.95455348,  0.45642527, -0.79206917, -0.48178957,
         1.24419512],
       [ 0.33625429, -1.04761023, -0.02921912, -0.63406093, -0.37478096,
        -0.9885205 ],
       [ 0.58835063,  0.95455348, -0.5589931 , -1.01112605, -1.43748712,
         0.49995658],
       ...,
       [-0.29398656, -1.04761023, -0.34097326, -1.01112605, -1.43748712,
        -0.9885205 ],
       [-2.05866094,  0.95455348, -0.3549896 , -1.01112605, -1.43748712,
        -0.24428196],
       [-0.42003473, -1.04761023,  0.20424059, -0.33240884,  0.97451146,
        -0.9885205 ]])

In [139]:
## Adding the data to a dataframe

df_scaled_data = pd.DataFrame(scaled_data, columns=['customer_age', 'gender', 
'credit_limit', 'avg_utilization_ratio','total_revolving_bal','income_category'])

df_scaled_data.head()

,customer_age,gender,credit_limit,avg_utilization_ratio,total_revolving_bal,income_category
0,-0.167938,0.954553,0.456425,-0.792069,-0.481790,1.244195
1,0.336254,-1.047610,-0.029219,-0.634061,-0.374781,-0.988520
2,0.588351,0.954553,-0.558993,-1.011126,-1.437487,0.499957
3,-0.798179,-1.047610,-0.570491,1.718107,1.658383,-0.988520
4,-0.798179,0.954553,-0.416859,-1.011126,-1.437487,1.244195


In [140]:
## List with a number of clusters

k=list(range(1,11))

In [141]:
## List to store the inertia values

inertia=[]

In [142]:
## Creating a for loop to compute the inertia with each possible value of k

for i in k:
## 1. Creating a KMeans model using the loop counter for the n_clusters

    k_model = KMeans(n_clusters=i, random_state=0)
## 2. Fit the model to the data using `df_market_data_scaled`

    k_model.fit(df_filtered)
## 3. Appending the model.inertia_ to the inertia list

    inertia.append(k_model.inertia_)

In [143]:
## Creating a dictionary with the data to plot the Elbow curve

elbow_data={
    "k":k,
    "inertia":inertia
}

## Creating a DataFrame with the data to plot the Elbow curve

df_elbow=pd.DataFrame(elbow_data)

In [144]:
##Plot a line chart with all the inertia values computed with

df_elbow.hvplot.line(x="k",y="inertia",title="Elbow Curve",xticks=k)

:Curve   [k]   (inertia)

In [145]:
## Initializing the K-Means model using k = 3

model = KMeans(n_clusters=3, random_state=0)

## Fit the K-Means model using the scaled data

model.fit(scaled_data)

KMeans(n_clusters=3, random_state=0)

In [146]:
## Finding the center for the clusters

model.cluster_centers_

array([[ 0.05802802,  0.94672234,  1.92688888, -0.84654194, -0.03477218,
         1.11223105],
       [ 0.02778185, -1.04202799, -0.50418049,  0.32824024, -0.02368798,
        -0.81421928],
       [-0.06565567,  0.95455348, -0.24735748, -0.03593033,  0.04888057,
         0.56601742]])

In [147]:
## Predicting the clusters to group

cluster_predictions=model.predict(scaled_data)
print(cluster_predictions)

[2 1 2 ... 1 2 1]


In [148]:
## Creating a copy of the dataset scaled to save the clusters

df_scaled_data_copy = df_scaled_data.copy()

In [149]:
## Adding the cluster prediction's column

df_scaled_data_copy['cluster_prediction']=cluster_predictions

df_scaled_data_copy.head()

,customer_age,gender,credit_limit,avg_utilization_ratio,total_revolving_bal,income_category,cluster_prediction
0,-0.167938,0.954553,0.456425,-0.792069,-0.481790,1.244195,2
1,0.336254,-1.047610,-0.029219,-0.634061,-0.374781,-0.988520,1
2,0.588351,0.954553,-0.558993,-1.011126,-1.437487,0.499957,2
3,-0.798179,-1.047610,-0.570491,1.718107,1.658383,-0.988520,1
4,-0.798179,0.954553,-0.416859,-1.011126,-1.437487,1.244195,2


In [150]:
## Plotting the model

scatter_plot = df_scaled_data_copy.hvplot.scatter(
    x="credit_limit", 
    y="gender", 
    by="cluster_prediction"
)

scatter_plot

:NdOverlay   [cluster_prediction]
   :Scatter   [credit_limit]   (gender)

In [151]:
## Plotting the model

scatter_plot = df_scaled_data_copy.hvplot.scatter(
    x="total_revolving_bal", 
    y="gender", 
    by="cluster_prediction"
)

scatter_plot

:NdOverlay   [cluster_prediction]
   :Scatter   [total_revolving_bal]   (gender)

In [152]:
## Plotting the model
scatter_plot = df_scaled_data_copy.hvplot.scatter(
    x="total_revolving_bal", 
    y="credit_limit", 
    by="cluster_prediction"
)

scatter_plot

:NdOverlay   [cluster_prediction]
   :Scatter   [total_revolving_bal]   (credit_limit)

### Optimize Clusters with Principal Component Analysis.

In [153]:
## Creating a PCA model instance and set `n_components=3`

pca=PCA(n_components=3)


In [154]:
## Using the PCA model with `fit_transform` to reduce to three principal components

data_pca=  data_scaled_pca=pca.fit_transform(scaled_data)

data_pca[:5]

array([[ 1.79474546, -0.1255963 , -0.16847559],
       [-0.74284627, -1.1609719 ,  0.39962444],
       [ 1.14616952, -1.25371031,  0.5852753 ],
       [-2.34772015,  1.48873708, -0.83460257],
       [ 1.61101698, -1.0819481 , -0.79038854]])

In [155]:
## Retrieving the explained variance

pca.explained_variance_ratio_

array([0.4172862 , 0.24347936, 0.16681549])

In [156]:
data_pca_df=pd.DataFrame(data_scaled_pca,columns=["PC1","PC2","PC3"])

data_pca_df.head()

,PC1,PC2,PC3
0,1.794745,-0.125596,-0.168476
1,-0.742846,-1.160972,0.399624
2,1.146170,-1.253710,0.585275
3,-2.347720,1.488737,-0.834603
4,1.611017,-1.081948,-0.790389


In [157]:
## List with a number of clusters

k = list(range(1, 11))

In [158]:
## List to store the inertia values

inertia=[]

In [159]:
## Creating a for loop to compute the inertia with each possible value of k

for i in k:
## 1. Creating a KMeans model using the loop counter for the n_clusters

    model=KMeans(n_clusters=i, random_state=1)
## 2. Fit the model to the data using `df_market_data_scaled`

    model.fit(data_pca_df)
## 3. Appending the model.inertia_ to the inertia list

    inertia.append(model.inertia_)

In [160]:
## Creating a dictionary with the data to plot the Elbow curve

elbow_data = {"k": k, "inertia": inertia}
## Fit the K-Means model using the scaled data

df_elbow = pd.DataFrame(elbow_data)

In [161]:
## Plot a line chart with all the inertia values computed with

elbow_plot_2= df_elbow.hvplot.line(x="k",y="inertia",title="Elbow Curve",xticks=k)
elbow_plot_2

:Curve   [k]   (inertia)

In [162]:
## Initializing the K-Means model using k = 4

model_pca = KMeans(n_clusters=4, random_state=0)

In [163]:
## Fit the K-Means model using the scaled data

model_pca.fit(data_pca_df)

KMeans(n_clusters=4, random_state=0)

In [164]:
## Predicting the clusters to group

PCA_clusters= model_pca.predict(data_pca_df)
PCA_clusters

array([0, 2, 0, ..., 2, 2, 1])

In [165]:
## Creating a copy of the dataset scaled to save the clusters

data_pca_df_copy=data_pca_df.copy()

In [166]:
## Addding the "PCA_clusters" column to the Dataframe

data_pca_df_copy['PCA_clusters']=PCA_clusters

data_pca_df_copy

,PC1,PC2,PC3,PCA_clusters
0,1.794745,-0.125596,-0.168476,0
1,-0.742846,-1.160972,0.399624,2
2,1.146170,-1.253710,0.585275,0
3,-2.347720,1.488737,-0.834603,1
4,1.611017,-1.081948,-0.790389,0
...,...,...,...,...
9010,-0.291583,1.119863,0.374237,3
9011,-0.415890,1.484601,-0.769277,3
9012,-0.566856,-2.213663,-0.206743,2
9013,0.854199,-1.525578,-2.059270,2


In [167]:
## Plotting the model

pca_scatter_plot=data_pca_df_copy.hvplot.scatter(
    x="PC1", 
    y="PC2", 
    by="PCA_clusters"
)
pca_scatter_plot

:NdOverlay   [PCA_clusters]
   :Scatter   [PC1]   (PC2)

In [168]:
## Calculating the Calinski-Harabasz coefficient to know if the clusters are well defined

kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(data_pca_df)
ch_score = calinski_harabasz_score(data_pca_df, labels)
print(f'Calinski-Harabasz Score: {ch_score}')

Calinski-Harabasz Score: 5057.260999634643


The clusters are well defined

In [169]:
## Adding the clusters to the first Dataframe

df_filtered_copy =df_filtered_1.copy()

df_filtered_copy['PCA_clusters']=PCA_clusters

df_filtered_copy.head(5)

,customer_age,gender,credit_limit,avg_utilization_ratio,total_revolving_bal,income_category,PCA_clusters
0,45,M,12691.0,0.061,777.0,$60K - $80K,0
1,49,F,8256.0,0.105,864.0,Less than $40K,2
2,51,M,3418.0,0.000,0.0,$80K - $120K,0
3,40,F,3313.0,0.760,2517.0,Less than $40K,1
4,40,M,4716.0,0.000,0.0,$60K - $80K,0


In [170]:
## Fist cluster

df_cluster_0= df_filtered_copy[df_filtered_copy['PCA_clusters'] == 0]
df_cluster_0.head(5)

,customer_age,gender,credit_limit,avg_utilization_ratio,total_revolving_bal,income_category,PCA_clusters
0,45,M,12691.0,0.061,777.0,$60K - $80K,0
2,51,M,3418.0,0.000,0.0,$80K - $120K,0
4,40,M,4716.0,0.000,0.0,$60K - $80K,0
6,51,M,34516.0,0.066,2264.0,$120K +,0
7,32,M,29081.0,0.048,1396.0,$60K - $80K,0


In [192]:
## Describing the data

grouped_0 = pd.DataFrame(df_cluster_0.groupby(['gender', 'income_category']).agg({
    'customer_age': ['min', 'max', 'mean', 'count'],
    'credit_limit': ['min', 'max', 'mean', 'count'],
    'avg_utilization_ratio': ['min', 'max', 'mean', 'count'],
    'total_revolving_bal': ['min', 'max', 'mean', 'count']
}).reset_index().sort_values(by='income_category'))

grouped_0

gender income_category customer_age                      credit_limit  \
                                  min max       mean count          min   
0      M         $120K +           26  62  47.880208   576       1438.3   
1      M     $40K - $60K           31  73  49.819277    83       4469.0   
2      M     $60K - $80K           26  63  46.761491   805       1438.3   
3      M    $80K - $120K           27  63  46.956569   921       1438.3   
4      M  Less than $40K           60  60  60.000000     1      14002.0   

                               avg_utilization_ratio                         \
       max          mean count                   min    max      mean count   
0  34516.0  23297.664757   576                 0.000  0.328  0.054342   576   
1  23981.0  14493.072289    83                 0.000  0.252  0.056904    83   
2  34516.0  14345.268199   805                 0.000  0.356  0.052576   805   
3  34516.0  21415.030945   921                 0.000  0.227  0.042232   921   
4  14002.0  14002.000000     1                 0.101  0.101  0.101000     1   

  total_revolving_bal                             
                  min     max         mean count  
0                 0.0  2517.0  1053.133681   576  
1                 0.0  1790.0   717.662651    83  
2                 0.0  2517.0   715.954037   805  
3                 0.0  2517.0   872.456026   921  
4              1413.0  1413.0  1413.000000     1

In [171]:
## Second cluster

df_cluster_1= df_filtered_copy[df_filtered_copy['PCA_clusters'] == 1]
df_cluster_1.head(5)

,customer_age,gender,credit_limit,avg_utilization_ratio,total_revolving_bal,income_category,PCA_clusters
3,40,F,3313.0,0.760,2517.0,Less than $40K,1
18,61,M,3193.0,0.788,2517.0,$40K - $60K,1
23,47,F,2492.0,0.626,1560.0,Less than $40K,1
25,41,F,7768.0,0.215,1669.0,Less than $40K,1
33,53,F,2650.0,0.562,1490.0,Less than $40K,1


In [193]:
## Describing the data

grouped_1 = pd.DataFrame(df_cluster_1.groupby(['gender', 'income_category']).agg({
    'customer_age': ['min', 'max', 'mean', 'count'],
    'credit_limit': ['min', 'max', 'mean', 'count'],
    'avg_utilization_ratio': ['min', 'max', 'mean', 'count'],
    'total_revolving_bal': ['min', 'max', 'mean', 'count']
}).reset_index().sort_values(by='income_category'))

grouped_1

gender income_category customer_age                      credit_limit  \
                                  min max       mean count          min   
0      F     $40K - $60K           26  65  47.164076   579       1438.3   
2      M     $40K - $60K           42  67  57.300000    60       1438.3   
1      F  Less than $40K           26  67  46.672916  2027       1438.3   
3      M  Less than $40K           26  70  52.673469    98       1438.3   

                              avg_utilization_ratio                         \
       max         mean count                   min    max      mean count   
0  15698.0  3624.891883   579                 0.127  0.999  0.552432   579   
2   3506.0  2330.981667    60                 0.431  0.953  0.716667    60   
1  15987.0  3393.082634  2027                 0.123  0.995  0.573902  2027   
3   7098.0  2797.665306    98                 0.244  0.984  0.647796    98   

  total_revolving_bal                             
                  min     max         mean count  
0               635.0  2517.0  1690.526770   579  
2               682.0  2517.0  1675.466667    60  
1               601.0  2517.0  1664.018747  2027  
3               653.0  2517.0  1712.020408    98

In [172]:
## Third cluster

df_cluster_2= df_filtered_copy[df_filtered_copy['PCA_clusters'] == 2]
df_cluster_2.head(5)

,customer_age,gender,credit_limit,avg_utilization_ratio,total_revolving_bal,income_category,PCA_clusters
1,49,F,8256.0,0.105,864.0,Less than $40K,2
14,57,F,2436.0,0.279,680.0,Less than $40K,2
21,62,F,1438.3,0.000,0.0,Less than $40K,2
22,41,M,4470.0,0.152,680.0,$40K - $60K,2
37,42,F,15433.0,0.000,0.0,Less than $40K,2


In [194]:
## Describing the data

grouped_2 = pd.DataFrame(df_cluster_2.groupby(['gender', 'income_category']).agg({
    'customer_age': ['min', 'max', 'mean', 'count'],
    'credit_limit': ['min', 'max', 'mean', 'count'],
    'avg_utilization_ratio': ['min', 'max', 'mean', 'count'],
    'total_revolving_bal': ['min', 'max', 'mean', 'count']
}).reset_index().sort_values(by='income_category'))

grouped_2

gender income_category customer_age                      credit_limit  \
                                  min max       mean count          min   
0      F     $40K - $60K           26  65  47.121891   402       1438.3   
2      M     $40K - $60K           26  65  44.558282   163       1438.3   
3      M    $80K - $120K           29  47  39.769231    26       1438.3   
1      F  Less than $40K           26  65  45.859761  1255       1438.3   
4      M  Less than $40K           26  65  44.945055    91       1438.3   

                              avg_utilization_ratio                         \
       max         mean count                   min    max      mean count   
0  23981.0  5536.255473   402                   0.0  0.468  0.070570   402   
2  12024.0  3999.519018   163                   0.0  0.373  0.011417   163   
3   3544.0  1678.026923    26                   0.0  0.000  0.000000    26   
1  15987.0  4136.809721  1255                   0.0  0.450  0.063273  1255   
4  15987.0  5199.479121    91                   0.0  0.434  0.059308    91   

  total_revolving_bal                            
                  min     max        mean count  
0                 0.0  1590.0  360.810945   402  
2                 0.0   713.0   34.828221   163  
3                 0.0     0.0    0.000000    26  
1                 0.0  1766.0  290.772112  1255  
4                 0.0  1047.0  248.241758    91

In [173]:
## Fourth cluster

df_cluster_3= df_filtered_copy[df_filtered_copy['PCA_clusters'] == 3]
df_cluster_3.head(5)

,customer_age,gender,credit_limit,avg_utilization_ratio,total_revolving_bal,income_category,PCA_clusters
5,44,M,4010.0,0.311,1247.0,$40K - $60K,3
8,37,M,22352.0,0.113,2517.0,$60K - $80K,3
9,48,M,11656.0,0.144,1677.0,$80K - $120K,3
10,42,M,6748.0,0.217,1467.0,$120K +,3
11,65,M,9095.0,0.174,1587.0,$40K - $60K,3


In [191]:
## Describing the data

grouped_3 = pd.DataFrame(df_cluster_3.groupby(['gender', 'income_category']).agg({
    'customer_age': ['min', 'max', 'mean', 'count'],
    'credit_limit': ['min', 'max', 'mean', 'count'],
    'avg_utilization_ratio': ['min', 'max', 'mean', 'count'],
    'total_revolving_bal': ['min', 'max', 'mean', 'count']
}).reset_index().sort_values(by='income_category'))

grouped_3

gender income_category customer_age                      credit_limit  \
                                  min max       mean count          min   
2      M         $120K +           27  61  46.549669   151       1438.3   
0      F     $40K - $60K           33  53  43.090909    33       8232.0   
3      M     $40K - $60K           26  67  42.525532   470       1438.3   
4      M     $60K - $80K           26  63  44.899497   597       1438.3   
5      M    $80K - $120K           29  63  45.894558   588       1438.3   
1      F  Less than $40K           33  34  33.500000     2      15487.0   
6      M  Less than $40K           26  65  38.252874    87       1775.0   

                               avg_utilization_ratio                         \
       max          mean count                   min    max      mean count   
2  23959.0   6059.858278   151                 0.105  0.856  0.396007   151   
0  23981.0  16870.000000    33                 0.061  0.274  0.129091    33   
3  23981.0   6173.707447   470                 0.072  0.983  0.350155   470   
4  28831.0   5922.711558   597                 0.085  0.976  0.413054   597   
5  34516.0   7655.289796   588                 0.073  0.994  0.355881   588   
1  15987.0  15737.000000     2                 0.119  0.125  0.122000     2   
6  15261.0   5829.425287    87                 0.073  0.781  0.362747    87   

  total_revolving_bal                             
                  min     max         mean count  
2               744.0  2517.0  1867.099338   151  
0              1454.0  2517.0  1990.181818    33  
3               574.0  2517.0  1579.919149   470  
4               535.0  2517.0  1746.998325   597  
5               584.0  2517.0  1783.663265   588  
1              1847.0  1992.0  1919.500000     2  
6               828.0  2517.0  1658.931034    87